In [22]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
import cv2
import random
import os
from PIL import Image, ImageEnhance
from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA
import pickle
import pandas as pd



In [23]:
male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data = np.load('female_data_small.npy',allow_pickle=True)

male_data_icdar=np.load('male_data_icdar.npy',allow_pickle=True)
female_data_icdar=np.load('female_data_icdar.npy',allow_pickle=True)


In [24]:

male_data_combined=np.concatenate((male_data,male_data_icdar[:186]))
female_data_combined=np.concatenate((female_data,female_data_icdar))
print(male_data_combined.shape)
print(female_data_combined.shape)


(418, 1420, 1000)
(417, 1420, 1000)


In [25]:
from skimage import feature, exposure

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (64, 128))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return (np.array(fd)).flatten()



In [26]:


N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    
    # if self.show_images:
    #     for cnt in contours : 
    #         cv2.drawContours(img2, [cnt], 0, (255, 0, 0), 1)  
            
    #     plt.imshow(img2, cmap='gray')
    return contours

def get_hinge_features(bw_image):
    # if self.is_binary:
    #     bw_image, _ = self.preprocess_binary_image(img_file, self.sharpness_factor, self.bordersize)
    # else:
    #     bw_image, _ = self.preprocess_image(img_file, self.sharpness_factor, self.bordersize)
    
    contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector


#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

def get_hinge_with_cold(bw_image):
    hinge=get_hinge_features(bw_image)
    cold=get_cold_features(bw_image)
    return np.concatenate([hinge, cold])
    

In [27]:

def make_model(male_data,female_data,get_feature,model_name,pca_flag=False,pca_number=50):
    male_feature=[]
    female_feature=[]
    for i,img in enumerate(male_data):
        male_feature.append(get_feature(img))

    for i,img in enumerate(female_data):        
        female_feature.append(get_feature(img))

    
    male_feature=np.array(male_feature)
    female_feature=np.array(female_feature)

    X=np.concatenate((male_feature,female_feature),axis=0)
    Y=np.concatenate((np.ones(male_feature.shape[0]),
                        np.zeros(female_feature.shape[0])),
                        axis=0)
    if pca_flag:
        pca=PCA(n_components=pca_number)
        X=pca.fit_transform(X)
        pickle.dump(pca,open(model_name+'_pca','wb'))

    
    X,Y=sklearn.utils.shuffle(X,Y)
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01,random_state=42) # 70% training and 30% test

    model = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

    model.fit(X, Y)
    
    #y_pred_cold=model.predict(X_test)
    #y_pred_prob_cold=model.predict_proba(X_test)
    #print(y_pred_prob_cold)
    #print("Accuracy:",metrics.accuracy_score(y_test, y_pred_cold))
    pickle.dump(model, open(model_name, 'wb'))

    
        

In [28]:
make_model(male_data,female_data,get_cold_features,"cold",True,50)

[LibSVM]..*.*
optimization finished, #iter = 1040
obj = -373.475037, rho = -0.619484
nSV = 182, nBSV = 12
Total nSV = 182
..*.*
optimization finished, #iter = 997
obj = -337.836705, rho = -0.704244
nSV = 192, nBSV = 7
Total nSV = 192
..*.*
optimization finished, #iter = 973
obj = -335.069918, rho = -0.844502
nSV = 181, nBSV = 7
Total nSV = 181
..*.*
optimization finished, #iter = 973
obj = -356.399649, rho = -0.705373
nSV = 178, nBSV = 10
Total nSV = 178
..*.*
optimization finished, #iter = 982
obj = -366.578345, rho = -0.545359
nSV = 189, nBSV = 12
Total nSV = 189
.*..*
optimization finished, #iter = 1111
obj = -438.583188, rho = 0.728214
nSV = 210, nBSV = 14
Total nSV = 210


In [16]:
make_model(male_data,female_data,get_hinge_features,"hinge",False)

[LibSVM].*.*
optimization finished, #iter = 717
obj = -1216.622500, rho = -2.620886
nSV = 195, nBSV = 123
Total nSV = 195
.*
optimization finished, #iter = 568
obj = -1184.358176, rho = -2.008217
nSV = 187, nBSV = 125
Total nSV = 187
.*.*
optimization finished, #iter = 846
obj = -1107.545181, rho = -2.038007
nSV = 186, nBSV = 106
Total nSV = 186
.*.*
optimization finished, #iter = 821
obj = -1214.044620, rho = -1.854121
nSV = 198, nBSV = 118
Total nSV = 198
.*.*
optimization finished, #iter = 711
obj = -1181.604039, rho = -2.041304
nSV = 189, nBSV = 119
Total nSV = 189
.*.*
optimization finished, #iter = 893
obj = -1478.424261, rho = 1.987140
nSV = 232, nBSV = 153
Total nSV = 232


In [17]:
make_model(male_data,female_data, HistogramofOrientedGradients,"hog",False)

[LibSVM].*.*
optimization finished, #iter = 665
obj = -249.693048, rho = -0.515540
nSV = 276, nBSV = 0
Total nSV = 276
.*.*
optimization finished, #iter = 636
obj = -246.952474, rho = -0.621603
nSV = 278, nBSV = 0
Total nSV = 278
.*.*
optimization finished, #iter = 658
obj = -257.898202, rho = -0.859037
nSV = 277, nBSV = 0
Total nSV = 277
.*.*
optimization finished, #iter = 663
obj = -250.110807, rho = -0.656442
nSV = 278, nBSV = 0
Total nSV = 278
.*.*
optimization finished, #iter = 626
obj = -244.142445, rho = -0.864131
nSV = 270, nBSV = 0
Total nSV = 270
.*.*
optimization finished, #iter = 810
obj = -316.868343, rho = 0.767954
nSV = 341, nBSV = 0
Total nSV = 341


In [29]:
make_model(male_data,female_data,get_hinge_with_cold,"hinge_with_cold",True,70)

[LibSVM].*.*
optimization finished, #iter = 838
obj = -292.193995, rho = -0.657542
nSV = 185, nBSV = 5
Total nSV = 185
.*.*
optimization finished, #iter = 825
obj = -292.809529, rho = -0.754919
nSV = 182, nBSV = 6
Total nSV = 182
.*.*
optimization finished, #iter = 829
obj = -313.190821, rho = -0.750453
nSV = 180, nBSV = 7
Total nSV = 180
..*.*
optimization finished, #iter = 908
obj = -301.533859, rho = -0.685267
nSV = 194, nBSV = 4
Total nSV = 194
..*.*
optimization finished, #iter = 914
obj = -288.268875, rho = -0.550923
nSV = 188, nBSV = 4
Total nSV = 188
..*.*
optimization finished, #iter = 1093
obj = -363.698825, rho = 0.742957
nSV = 229, nBSV = 9
Total nSV = 229


In [30]:
make_model(male_data_combined,female_data_combined,get_cold_features,"cold_combined",True,50)

[LibSVM]...*.*
optimization finished, #iter = 2873
obj = -1691.579911, rho = -0.480778
nSV = 456, nBSV = 117
Total nSV = 456
..*.*
optimization finished, #iter = 2565
obj = -1672.307732, rho = -0.761678
nSV = 440, nBSV = 125
Total nSV = 440
..*.*
optimization finished, #iter = 2637
obj = -1585.621568, rho = -0.526492
nSV = 436, nBSV = 113
Total nSV = 436
..*.*
optimization finished, #iter = 2484
obj = -1608.476493, rho = -0.799134
nSV = 432, nBSV = 117
Total nSV = 432
..*.*
optimization finished, #iter = 2633
obj = -1614.327416, rho = -0.758440
nSV = 442, nBSV = 120
Total nSV = 442
..*.*
optimization finished, #iter = 3215
obj = -2118.655778, rho = 0.676183
nSV = 533, nBSV = 157
Total nSV = 533


In [25]:
make_model(male_data_combined,female_data_combined,get_hinge_features,"hinge_combined",False)

[LibSVM].*.*
optimization finished, #iter = 1787
obj = -3400.675937, rho = -2.738917
nSV = 470, nBSV = 344
Total nSV = 470
.*.*
optimization finished, #iter = 1689
obj = -3368.135728, rho = -2.326981
nSV = 473, nBSV = 351
Total nSV = 473
.*.*
optimization finished, #iter = 1521
obj = -3380.725844, rho = -1.978955
nSV = 465, nBSV = 346
Total nSV = 465
.*.*
optimization finished, #iter = 1586
obj = -3355.656825, rho = -2.074608
nSV = 461, nBSV = 350
Total nSV = 461
.*.*
optimization finished, #iter = 1635
obj = -3274.664076, rho = -1.849740
nSV = 453, nBSV = 331
Total nSV = 453
.*.*
optimization finished, #iter = 2054
obj = -4220.140859, rho = 2.242158
nSV = 577, nBSV = 439
Total nSV = 577


In [26]:
make_model(male_data_combined,female_data_combined, HistogramofOrientedGradients,"hog_combined",False)

[LibSVM].*.*
optimization finished, #iter = 1698
obj = -712.614730, rho = -0.756568
nSV = 635, nBSV = 0
Total nSV = 635
.*.*
optimization finished, #iter = 1752
obj = -750.392186, rho = -0.878135
nSV = 641, nBSV = 0
Total nSV = 641
.*.*
optimization finished, #iter = 1733
obj = -746.051461, rho = -1.117814
nSV = 636, nBSV = 1
Total nSV = 636
.*.*
optimization finished, #iter = 1798
obj = -766.260864, rho = -0.718146
nSV = 647, nBSV = 0
Total nSV = 647
.*.*
optimization finished, #iter = 1712
obj = -749.582123, rho = -1.043393
nSV = 635, nBSV = 0
Total nSV = 635
.*.*
optimization finished, #iter = 2220
obj = -958.418907, rho = 1.092036
nSV = 785, nBSV = 1
Total nSV = 785


In [31]:
make_model(male_data_combined,female_data_combined,get_hinge_with_cold,"hinge_with_cold_combined",True,70)

[LibSVM]..*.*
optimization finished, #iter = 2503
obj = -1399.067212, rho = -0.766717
nSV = 446, nBSV = 82
Total nSV = 446
..*.*
optimization finished, #iter = 2603
obj = -1384.150337, rho = -0.752579
nSV = 451, nBSV = 90
Total nSV = 451
..*.*
optimization finished, #iter = 2616
obj = -1435.911705, rho = -0.913676
nSV = 449, nBSV = 90
Total nSV = 449
..*.*
optimization finished, #iter = 2484
obj = -1411.219379, rho = -0.392351
nSV = 448, nBSV = 91
Total nSV = 448
..*.*
optimization finished, #iter = 2434
obj = -1414.923404, rho = -0.377074
nSV = 440, nBSV = 90
Total nSV = 440
..*.*
optimization finished, #iter = 3083
obj = -1820.415500, rho = 0.593315
nSV = 543, nBSV = 123
Total nSV = 543


In [39]:
loaded_model = pickle.load(open('hinge_with_cold', 'rb'))
pca= pickle.load(open('hinge_with_cold_pca','rb'))

troll=[]
for i in range(100):
    hinge=get_hinge_features(female_data[i])
    cold=get_cold_features(female_data[i])
    data=np.concatenate((hinge,cold))
    troll.append(data)
    
troll=np.array(troll)
troll=pca.transform(troll)
print(troll.shape)
y=loaded_model.predict(troll)



(100, 70)


In [40]:
print(y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
